# 4.3 오버피팅과 정규화 (Overfitting and Regularization)

머신러닝 모델
과적합(Overfitting)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [3]:
EPOCHS = 50
BATCH_SIZE = 64

## 데이터셋에 노이즈 추가하기

![original.png](./assets/original.png)

![horizontalflip.png](./assets/horizontalflip.png)

In [4]:
train_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
                   train=True,
                   download=True,
                   transform=transforms.Compose([
                       transforms.RandomHorizontalFlip(),
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.FashionMNIST('./.data',
                   train=False, 
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=BATCH_SIZE, shuffle=True)

Extracting ./.data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./.data/FashionMNIST/raw


Extracting ./.data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./.data/FashionMNIST/raw




Extracting ./.data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./.data/FashionMNIST/raw


Extracting ./.data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./.data/FashionMNIST/raw
Processing...
Done!


/Users/keon/projects/3-min-pytorch/env/lib/python3.8/site-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


## 뉴럴넷으로 Fashion MNIST 학습하기

입력 `x` 는 `[배치크기, 색, 높이, 넓이]`로 이루어져 있습니다.
`x.size()`를 해보면 `[64, 1, 28, 28]`이라고 표시되는 것을 보실 수 있습니다.
Fashion MNIST에서 이미지의 크기는 28 x 28, 색은 흑백으로 1 가지 입니다.
그러므로 입력 x의 총 특성값 갯수는 28 x 28 x 1, 즉 784개 입니다.

우리가 사용할 모델은 3개의 레이어를 가진 뉴럴네트워크 입니다. 

In [5]:
class Net(nn.Module):
    def __init__(self, dropout_p=0.2):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        # 드롭아웃 확률
        self.dropout_p = dropout_p

    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        # 드롭아웃 추가
        x = F.dropout(x, training=self.training,
                      p=self.dropout_p)
        x = F.relu(self.fc2(x))
        # 드롭아웃 추가
        x = F.dropout(x, training=self.training,
                      p=self.dropout_p)
        x = self.fc3(x)
        return x

## 모델 준비하기 

`to()` 함수는 모델의 파라미터들을 지정한 곳으로 보내는 역할을 합니다.
일반적으로 CPU 1개만 사용할 경우 필요는 없지만,
GPU를 사용하고자 하는 경우 `to("cuda")`로 지정하여 GPU로 보내야 합니다.
지정하지 않을 경우 계속 CPU에 남아 있게 되며 빠른 훈련의 이점을 누리실 수 없습니다.

최적화 알고리즘으로 파이토치에 내장되어 있는 `optim.SGD`를 사용하겠습니다.

In [6]:
model        = Net(dropout_p=0.2).to(DEVICE)
optimizer    = optim.SGD(model.parameters(), lr=0.01)

## 학습하기

In [7]:
def train(model, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

## 테스트하기


In [8]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            test_loss += F.cross_entropy(output, target,
                                         reduction='sum').item()
            
            # 맞춘 갯수 계산
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

## 코드 돌려보기

자, 이제 모든 준비가 끝났습니다. 코드를 돌려서 실제로 훈련이 되는지 확인해봅시다!

In [9]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
          epoch, test_loss, test_accuracy))

[1] Test Loss: 0.6536, Accuracy: 76.92%
[2] Test Loss: 0.5359, Accuracy: 80.72%
[3] Test Loss: 0.4977, Accuracy: 82.18%
[4] Test Loss: 0.4584, Accuracy: 83.49%
[5] Test Loss: 0.4421, Accuracy: 84.22%
[6] Test Loss: 0.4245, Accuracy: 84.91%
[7] Test Loss: 0.4205, Accuracy: 84.82%
[8] Test Loss: 0.4067, Accuracy: 85.32%
[9] Test Loss: 0.4024, Accuracy: 85.54%
[10] Test Loss: 0.3923, Accuracy: 85.82%
[11] Test Loss: 0.3875, Accuracy: 86.22%
[12] Test Loss: 0.3772, Accuracy: 86.19%
[13] Test Loss: 0.3807, Accuracy: 86.11%
[14] Test Loss: 0.3762, Accuracy: 86.33%
[15] Test Loss: 0.3636, Accuracy: 86.70%
[16] Test Loss: 0.3699, Accuracy: 86.29%
[17] Test Loss: 0.3625, Accuracy: 86.92%
[18] Test Loss: 0.3567, Accuracy: 87.06%
[19] Test Loss: 0.3515, Accuracy: 87.41%
[20] Test Loss: 0.3490, Accuracy: 87.50%
[21] Test Loss: 0.3572, Accuracy: 86.86%
[22] Test Loss: 0.3504, Accuracy: 87.10%
[23] Test Loss: 0.3465, Accuracy: 87.48%
[24] Test Loss: 0.3383, Accuracy: 87.60%
[25] Test Loss: 0.3347, A